# XGBoost in Python

This notebook tries to interprete [Introduction to Boosted Trees](https://xgboost.readthedocs.io/en/latest/tutorials/model.html) with Python codes.

We'll use quared error as loss function here.

In [245]:
# Prepare the input data
from typing import NamedTuple, Optional

class Person(NamedTuple):
    age: float
    gender: bool # True for male, False for female
    likes_game: Optional[float] = None

In [246]:
# Define the model
from typing import NamedTuple, Union, Any, Dict, Optional

class NonLeaf(NamedTuple):
    attribute: str
    children: dict
    default_score: float = 0.0

class Leaf(NamedTuple):
    score: float = 0.0

Model = Union[Leaf, NonLeaf]
        
class BoostModel(NamedTuple):
    models:List[Model] = [Leaf(0.0)]

def find_model_by_real_value(value:float, models:Dict[Any, Model]) -> Optional[float]:
    for x in models:
        if value <= x:
            return x
    return None

def predict_with_simple_model(input:Person, model: Model) -> Any:
    if isinstance(model, Leaf):
        return model.score
    else:
        key = getattr(input, model.attribute)
        if isinstance(key, float):
            key = find_model_by_real_value(key, model.children)
        
        if key in model.children:
            sub_model = model.children[key]
            return predict_with_simple_model(input, sub_model)
        else:
            return model.default_score

# Entrance for the prediction
def predict(input: Person, model: BoostModel) -> float:
    scores = [predict_with_simple_model(input, m) for m in model.models]
    return sum(scores) / len(model.models)

In [247]:
# Train model
from typing import Dict, List, Any, Tuple, Optional
import math

# Hyper parameters
alpha = 0.1 # originally the lambda in the post
gamma = 0.1

def get_label_values(inputs: List[Person]) -> List[float]:
    return [getattr(p, 'likes_game') for p in inputs]

def get_predicted_values(current_model: BoostModel, inputs:List[Person]) -> List[float]:
    return [predict(p, current_model) for p in inputs]

# use squared error as loss, the gi, hi will be:
def gi(y:float, y_hat:float) -> float:
    return 2 * y_hat - 2 * y

def hi(y:float, y_hat:float) -> float:
    return 2

# get the value for the objective function, with the given model and inputs
def objective_value(current_model:BoostModel, inputs:List[Person]) -> float:
    ys = get_label_values(inputs)
    y_hats = get_predicted_values(current_model, inputs)
    G = sum([gi(y, y_hat) for (y, y_hat) in zip(ys, y_hats)])
    H = sum([hi(y, y_hat) for (y, y_hat) in zip(ys, y_hats)])
    return G * G / (H + alpha)

# get the score of a Leaf node
def score_value(current_model:BoostModel, inputs:List[Person]) -> float:
    ys = get_label_values(inputs)
    y_hats = get_predicted_values(current_model, inputs)
    G = sum([gi(y, y_hat) for (y, y_hat) in zip(ys, y_hats)])
    H = sum([hi(y, y_hat) for (y, y_hat) in zip(ys, y_hats)])
    return - G / (H + alpha)

def gain_of_partitions(current_model: BoostModel, partitions:Dict[Any, List[Person]], prev_score:float) -> float:
    partition_scores = sum([objective_value(current_model, partition) for _, partition in partitions.items()])
    return (partition_scores - prev_score) / 2 - gamma

# partition the input data with a given attribute
def partition_by(inputs:List[Person], attri:str) -> Dict[Any, List[Person]]:
    partitions = {}
    for person in inputs:
        v = getattr(person, attri)
        if v in partitions:
            partitions[v].append(person)
        else:
            partitions[v] = [person]
    return partitions

# partition the input data with a given attribute, which has data type of float
def partition_by_real_value(sorted_inputs:List[Person], attri:str, i:int) -> Dict[Any, List[Person]]:
    partition_value = getattr(sorted_inputs[i], attri)
    left = [person for (j, person) in enumerate(sorted_inputs) if j <= i]
    right = [person for (j, person) in enumerate(sorted_inputs) if j > i]
    partitions = {partition_value: left, float("inf"): right}
    return partitions

# build the tree layer by layer according to the gain of the new layers
# stop building if there is no extra attributes or no further positive gain
def build_tree_layer(current_model: BoostModel, inputs:List[Person], ignored_attris:List[str]) -> Model:
    # calculating score on the original/parent layer
    original_score = objective_value(current_model, inputs)
    
    # iterating each attributes of Person to get the partition with maximum gain
    # return a Leaf Model if no such partition can be found
    attributes = [attri for attri in Person._fields if attri not in ignored_attris]
    max_gain = 0
    best_partition = None
    partition_attri = None
    for attri in attributes:
        values = [getattr(input, attri) for input in inputs]
        is_real_value = isinstance(getattr(inputs[0], attri),float)
        if is_real_value:
            # sort by value in natural order
            sorted_inputs = sorted(inputs, key = lambda person: getattr(person, attri))
            for i in range(len(sorted_inputs)):
                partitions = partition_by_real_value(inputs, attri, i)
                gain = gain_of_partitions(current_model, partitions, original_score)
                if gain > max_gain:
                    max_gain = gain
                    best_partition = partitions
                    partition_attri = attri
        else:
            partitions = partition_by(inputs, attri)
            gain = gain_of_partitions(current_model, partitions, original_score)
            if gain > max_gain:
                max_gain = gain
                best_partition = partitions
                partition_attri = attri
    
    score = score_value(current_model, inputs)
    if not best_partition:
        return Leaf(score)
    else:
        # build the next layer for each children
        next_layer = {k: build_tree_layer(current_model, partition, ignored_attris + [partition_attri]) for k, partition in best_partition.items()}
        return NonLeaf(partition_attri, next_layer, score)

# train a new model/tree with the given inputs and current BoostModel
def build_tree_model(inputs:List[Person], current_model: BoostModel) -> Optional[Model]:
    # get the actual values and predicted scores with current model
    ys = [getattr(p, 'likes_game') for p in inputs]
    y_hats = [predict(p, current_model) for p in inputs]
    
    # build the tree layer by layer, by finding the partition with maximum gain
    return build_tree_layer(current_model, inputs, ['likes_game'])

# Entrance for the traning
def train(inputs:List[Person], num_trees:int) -> BoostModel:
    # a initial model will output score 0.0 for every input
    model = BoostModel(models = [Leaf(0.0)])
    for _ in range(num_trees):
        new_model = build_tree_model(inputs, model)
        if isinstance(new_model, Leaf):
            break
        else:
            model.models.append(new_model)
    return model

In [248]:
inputs = [
    Person(14.0, True, 1.0),
    Person(14.0, False, 0.0),
    Person(15.0, False, 0.0),
    Person(18.0, True, 1.0),
    Person(32.0, False, 0.0),
    Person(56.0, True, 0.0),
    Person(58.0, False, 0.0)
]

model = train(inputs, 2)

In [251]:
# debug the model:
for m in model.models:
    print(m)

Leaf(score=0.0)
NonLeaf(attribute='gender', children={True: NonLeaf(attribute='age', children={18.0: Leaf(score=0.9756097560975611), inf: Leaf(score=-0.0)}, default_score=0.6557377049180328), False: Leaf(score=-0.0)}, default_score=0.28368794326241137)
NonLeaf(attribute='gender', children={True: NonLeaf(attribute='age', children={18.0: Leaf(score=0.49970255800118984), inf: Leaf(score=-0.0)}, default_score=0.33586565373850463), False: Leaf(score=-0.0)}, default_score=0.14530358069538143)


In [252]:
print(predict(Person(14.0, False), model)) # a 14 years' girl
print(predict(Person(18.0, True), model)) # a 14 years' boy
print(predict(Person(40.0, False), model)) # a 40 years' woman
print(predict(Person(32.0, True), model)) # a 32 years' man

0.0
0.4917707713662503
0.0
0.0
